In [16]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer

def init_random_num(qc, reg):
    """Randomly initializes an n-qubit quantum register and prints the register label and the binary number."""
    n = len(reg)  # number of qubits in the register
    binary_representation = ''
    
    for qubit in range(n): # set to n-1 to leave MSB as 0
        # Randomly apply X gate to flip the qubit with 50% probability
        if np.random.rand() > 0.5:
            qc.x(reg[qubit])  # Apply X gate to the quantum circuit
            binary_representation = '1' + binary_representation
        else:
            binary_representation = '0' + binary_representation
            
    #binary_representation = '0' + binary_representation #toggle this on when MSB set to 0
    decimal_value = int(binary_representation, 2)
    
    # Print the label and the binary number created
    print(f"{reg.name}: {binary_representation} (decimal: {decimal_value})") # choose to leave MSB as 0
    
    return qc

In [29]:
""" Function """
def square(qc, a, mult_reg, c): 
    """ Info
    This function find the square of an input register 'a' and stores the product in mult reg.

    Parameters:
    - qc: QuantumCircuit on which the addition is performed
    - a: QuantumRegister for the number we wish to square
    - mult_reg: The register in with the product of 'a' and 'b' is stored
    c - ancillaries carry qubits

    Returns: 
    qc - 
    
    """

    def ctrl_add_cout(qc, a, b, ctrl_value, c, c_out): #This is for the last add
        """
        Takes in a quantum circuit and two quantum registers (a and b),
        performs a controlled addition WITH OUTPUT CARRY, and returns the updated circuit.
        
        Parameters:
        - qc: QuantumCircuit on which the addition is performed
        - a: QuantumRegister for the first number
        - b: QuantumRegister for the second number
        - ctrl: Control qubit that determines if the ADD is performed
        - c: QuantumRegister for carry qubits
        - c_out: this is given as an individual qubit not a register, it holds the carry out
        
        Returns:
        - QuantumCircuit: The updated quantum circuit with the addition operation applied.
        """
    
        n = len(a)  # number of qubits per register
            
       # First seg TempAND
        qc.ccx(a[0], b[0], c[0]) 
    
        # ctrled Gidney adder
        for i in range(1,n-1):
            qc.cx(c[i-1], a[i])
            qc.cx(c[i-1], b[i])
            qc.ccx(a[i], b[i], c[i])
            qc.cx(c[i-1], c[i])

        # Last Segment
        qc.cx(c[n-2], a[n-1])
        qc.cx(c[n-2], b[n-1])

        # need a 3- bit controlled gate for carry out
        qc.ccx(ctrl_value, a[n-1], c[n-1])
        qc.ccx(c[n-1], b[n-1], c_out) 
        qc.ccx(ctrl_value, a[n-1], c[n-1])
        
        qc.ccx(ctrl_value, c[n-2], c_out) # final gate on c_out
    
        qc.ccx(ctrl_value, a[n-1], b[n-1])
        qc.cx(c[n-2],a[n-1])
        qc.cx(c[n-2],b[n-1])
    
        #un compute + last bits
        for i in range(n-2,0,-1): # i goes from n-2 to 1
            #uncompute
            qc.cx(c[i-1], c[i])
            qc.ccx(a[i], b[i], c[i])
    
            qc.ccx(ctrl_value, a[i], b[i])
            qc.cx(c[i-1], a[i])
            qc.cx(c[i-1], b[i])
    
        # End of first
        qc.ccx(a[0], b[0], c[0])
        qc.ccx(ctrl_value, a[0], b[0])
        
        return qc

    n = len(a) # this will be used to determine the number of bits in an addition
    #apply the first toffoli array 

    #we will use the first ancillary bit to copy out individual qubits from 'a'
    qc.cx(a[0], c[0]) 
    
    for i in range(0, n):
        qc.ccx(c[0],a[i], mult_reg[i])
        
    qc.cx(a[0], c[0]) #rest the ancilla for the next copy
    
    for i in range(1,n):
        qc.cx(a[i], c[0]) 
        qc = ctrl_add_cout(qc, a, mult_reg[i:i+n], c[0], c[1:], mult_reg[i+n]) #(qc, a, b, ctrl_value, c, c_out), n+1 qubits in t
        qc.cx(a[i], c[0]) 
        
    return qc

In [32]:
""" MAIN, This does a x b"""

# Number of bits
n = 5

ancillaries = QuantumRegister(n+1, 'ancillaries')
a = QuantumRegister(n, 'a')
mult_reg = QuantumRegister(2*n, 'mult_reg') 
#c_out = QuantumRegister(1, 'c_out')

# Define a classical register with 2n bits (n for a and n for b)
result = ClassicalRegister( n + 2*n, 'result')

qc = QuantumCircuit(a, ancillaries, result, mult_reg)

"""initialize the quantum numbers"""
qc = init_random_num(qc, a)

#for i in range (0, n):
#    qc.x(a[i])

#apply the square operation
qc = square(qc, a, mult_reg, ancillaries)


# Measure the qubits into the classical register
qc.measure(a, result[:n])      # Measure 'a' into the first n classical bits
qc.measure(mult_reg, result[n:2*n+n]) # Measure the multiplication result into the next n 
#qc.measure(ancillaries,[])

# For execution
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
sim_result = simulator.run(compiled_circuit).result()
counts = sim_result.get_counts()

print(counts)

print("Measurement results:")
for bitstring, count in counts.items():
    # bitstring is of the form 'result' where the first n bits are for 'a' and the last n bits are for 'b'
    a_result = bitstring[-n:]  # Last n bits for b
    mult_result = bitstring[-3*n:-n]  # First 2n bits for a x b
    a_decimal_value = int(a_result, 2)
    mult_decimal_value = int(mult_result, 2)

    print(f"a = {a_result} (decimal: {a_decimal_value}), a^2 = {mult_result} (decimal: {mult_decimal_value})")
    print(f"Check: {a_decimal_value}^2 = {a_decimal_value**2}")


{'111100000111111': 1024}
Measurement results:
a = 11111 (decimal: 31), a^2 = 1111000001 (decimal: 961)
Check: 31^2 = 961


In [25]:
list=[]
mult_reg=[]
list_a = ['a','b','c','d']
num_add = len(list_a)

for w in range(0,10):
    list.append(w)

print(list[1:])
print(list[-4:])

for w in range(0,19):
    mult_reg.append(-w)

print(list[-4:-1])
n=len(list)
for i in range(1,num_add-1):
    print(str(list))
    print("\n")
    print(mult_reg[i:i+n])
    print("\n")
    print(list_a[i]+"/n")

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[6, 7, 8, 9]
[6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10]


b/n
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


[-2, -3, -4, -5, -6, -7, -8, -9, -10, -11]


c/n
